<a href="https://colab.research.google.com/github/priyasakila/Priya.v/blob/main/flight%20delay%20predictin%20task%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score

In [4]:
dataset=pd.read_csv("/content/flight_data_2018_to_2022.csv")

In [5]:
dataset.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40669 entries, 0 to 40668
Columns: 120 entries, Year to Unnamed: 119
dtypes: float64(83), int64(16), object(21)
memory usage: 37.2+ MB


In [8]:
dataset = dataset.drop('Unnamed: 119', axis=1)
dataset.isnull().sum()

Year                    0
Quarter                 0
Month                   0
DayofMonth              0
DayOfWeek               0
                    ...  
Div5TotalGTime      40669
Div5LongestGTime    40669
Div5WheelsOff       40669
Div5TailNum         40669
Duplicate               1
Length: 119, dtype: int64

In [9]:
dataset=dataset[["Month","DayofMonth","Origin","Dest","CRSArrTime","DepDelay","ArrDelay"]]
dataset.isnull().sum()

Month            0
DayofMonth       0
Origin           0
Dest             1
CRSArrTime       1
DepDelay      1748
ArrDelay      1802
dtype: int64

In [10]:
dataset[dataset.isnull().any(axis=1)].head(10)

,Month,DayofMonth,Origin,Dest,CRSArrTime,DepDelay,ArrDelay
0,1,6,FLL,LGA,1419.0,NaN,NaN
30,1,6,DTW,BNA,2216.0,NaN,NaN
53,1,6,ATL,ORF,2012.0,-2.0,NaN
75,1,6,CVG,BOS,1903.0,NaN,NaN
106,1,6,IAD,SEA,2019.0,-1.0,NaN
148,1,6,MIA,LGA,2307.0,NaN,NaN
157,1,6,ATL,BNA,1646.0,NaN,NaN
158,1,6,BNA,ATL,2010.0,NaN,NaN
201,1,6,EWR,ATL,829.0,NaN,NaN
242,1,6,TPA,LGA,2053.0,NaN,NaN


In [11]:
dataset['DepDelay'].mode()

0   -3.0
Name: DepDelay, dtype: float64

In [12]:
dataset = dataset.fillna({'ArrDelay':1})
dataset = dataset.fillna({"DepDelay":0})
dataset.iloc[177:185]

,Month,DayofMonth,Origin,Dest,CRSArrTime,DepDelay,ArrDelay
177,1,6,CHS,ATL,1148.0,-8.0,-23.0
178,1,6,AUS,DTW,930.0,-9.0,-21.0
179,1,6,TPA,LGA,1743.0,-5.0,-24.0
180,1,6,SJU,ATL,1110.0,-6.0,-24.0
181,1,6,ATL,STX,1440.0,86.0,77.0
182,1,6,STX,ATL,1913.0,61.0,36.0
183,1,6,STT,ATL,1747.0,-7.0,-27.0
184,1,6,SJU,JFK,2159.0,-8.0,-40.0


In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['Dest'] = le.fit_transform(dataset['Dest'])
dataset['Origin'] = le.fit_transform(dataset['Origin'])

In [14]:
dataset.head(5)

,Month,DayofMonth,Origin,Dest,CRSArrTime,DepDelay,ArrDelay
0,1,6,42,68,1419.0,0.0,1.0
1,1,6,3,42,1821.0,-4.0,-1.0
2,1,6,42,3,2127.0,-2.0,-12.0
3,1,6,42,102,1227.0,-5.0,-15.0
4,1,6,3,61,1142.0,-4.0,-11.0


In [15]:
dataset['Origin'].unique()

array([ 42,   3,  61, 103,  82, 102,  10,  59,  68, 110, 131, 109, 126,
       117,   7,  62,  12, 107, 112,  74,  77,  32,  97,  73,  22,  18,
        36,  15,  95,  24,  63,  65,  94,  28,  92,  43,  66,  30,  76,
       116, 106,  91, 111,  23,  98,  96,  56,  20,  27, 101,  60, 119,
        72,  69,  11,  83, 127, 122,  48,  80,   5, 123,  86,  29,  16,
        71,  53,  33,  25, 120, 108, 130,  55,  57, 115,  93, 121,  85,
        21,  75,  90,  89, 113,  78,   2,  38,  54,   8, 129,  88,  51,
        19,  17,  31,  37,  58,  99, 105,  49, 125,  44,   4,  45,  41,
        79,  50,  67,  13,   6, 104,  46, 114,  26, 100, 118,   1,  81,
       124,  34,  52,  70,  87,  64,   0, 128,  35,  14,  39,  47,  84,
        40,   9])

In [16]:
dataset = pd.get_dummies(dataset,columns=['Origin', 'Dest'])
dataset = dataset.head()

In [17]:
x = dataset.iloc[:,0:8].values
y = dataset.iloc[:,8:9].values

In [18]:
x

array([[ 1.000e+00,  6.000e+00,  1.419e+03,  0.000e+00,  1.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00],
       [ 1.000e+00,  6.000e+00,  1.821e+03, -4.000e+00, -1.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00],
       [ 1.000e+00,  6.000e+00,  2.127e+03, -2.000e+00, -1.200e+01,
         0.000e+00,  0.000e+00,  0.000e+00],
       [ 1.000e+00,  6.000e+00,  1.227e+03, -5.000e+00, -1.500e+01,
         0.000e+00,  0.000e+00,  0.000e+00],
       [ 1.000e+00,  6.000e+00,  1.142e+03, -4.000e+00, -1.100e+01,
         0.000e+00,  0.000e+00,  0.000e+00]])

In [19]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder()
z=oh.fit_transform(x[:,4:5]).toarray()
t=oh.fit_transform(x[:,5:6]).toarray()


In [20]:
z

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [21]:
t

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [22]:
x=np.delete(x,[4,5],axis=1)